In [24]:
import wikipediaapi as wiki
from tqdm.notebook import tqdm
import os
import json

In [3]:
wiki_wiki = wiki.Wikipedia("RUG NLP Q&A", 'en')

In [15]:
# Get all pages mentioned in this url https://en.wikipedia.org/wiki/Wikipedia:Popular_pages
popular_pages = wiki_wiki.page("Wikipedia:Popular_pages")
popular_pages_links = popular_pages.links

In [23]:
# Get the text of all the pages as specified in the links above
# Save the text in a file with the name of the page as the file name, save it in data/raw
for page in tqdm(popular_pages_links):
    page = wiki_wiki.page(page)

    # If the file already exists, skip it
    if os.path.exists(f"data/raw/{page.title}.txt"):
        continue
    
    # Account for pages with / in their name by replacing it with _
    file_name = page.title.replace("/", "_")

    # replace ? with _ in the file name
    file_name = file_name.replace("?", "_")

    with open(f"data/raw/{file_name}.txt", "w") as f:
        f.write(page.text)


  0%|          | 0/1079 [00:00<?, ?it/s]

In [26]:
# Load the text of all the pages in data/raw and store it in a json file in data/bronze
data = {}
for file in tqdm(os.listdir("data/raw")):
    with open(f"data/raw/{file}", "r") as f:
        data[file] = f.read()

with open("data/bronze/data.json", "w") as f:
    json.dump(data, f)

  0%|          | 0/1025 [00:00<?, ?it/s]